In [41]:
# ticker로 종가 가져오는 함수 만들기 두번째
def get_close_price_btn(ticker_tuple,date,duration):
  ticker, market = ticker_tuple
  # 20240727 새 함수 작성
  import yfinance as yf
  import pykrx.stock as krx
  import pandas as pd
  from datetime import datetime, timedelta
  import numpy as np
  import pykrx.stock as krx

  reference_date = datetime.strptime(date, '%Y-%m-%d')

  # duration 전후 데이터 가져오기
  start_date = (reference_date - timedelta(days=duration*365))
  end_date = (reference_date + timedelta(days=duration*365))

  if market=='yahoo': 
    # Yahoo Finance에서 데이터 가져오기
    try:
      stock = yf.Ticker(ticker)
      data = stock.history(start=start_date, end=end_date)
      df=pd.DataFrame(data['Close'])

      # 인덱스 포맷 변경
      df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')

      # 기준일의 종가(값이 없을 시 직전 영업일 값으로)
      ref_price = df['Close'].loc[:reference_date.strftime('%Y-%m-%d')].ffill().iloc[-1]

      # 정규화하여 수정주가 칼럼 추가
      df['Adjusted_Close'] = (df['Close'] / ref_price) * 100

      # 6개월 역사적 변동성 계산 (일간 수익률 기준, 연율화)
      df['Returns'] = df['Close'].pct_change()
      df['Volatility_6M'] = df['Returns'].rolling(window=126).std() * np.sqrt(252)

      # 칼럼 이름 변경
      df.columns = [ticker, ticker+'수정주가', '수익율', ticker+'변동성']

      return df
      
    except KeyError: # 2024년 7월26일 수정
      print('현재가가 없는 종목입니다.')
      return np.nan

  elif market=='krx':
    # pykrx로 데이터 가져오기 시도
    # pykrx로 데이터 가져오기 시도
    # 새로운 날짜를 문자열로 변환
    prev_date = start_date.strftime('%Y%m%d')
    date = end_date.strftime('%Y%m%d')
    data = krx.get_market_ohlcv(prev_date, date, ticker)
    df=pd.DataFrame(data['종가'])

    # 인덱스 포맷 변경
    df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')

    # 기준일의 종가(값이 없을 시 직전 영업일 값으로)
    ref_price = df['종가'].loc[:reference_date.strftime('%Y-%m-%d')].ffill().iloc[-1]

    # 정규화하여 수정주가 칼럼 추가
    df['Adjusted_Close'] = (df['종가'] / ref_price) * 100

    # 6개월 역사적 변동성 계산 (일간 수익률 기준, 연율화)
    df['Returns'] = df['종가'].pct_change()
    df['Volatility_6M'] = df['Returns'].rolling(window=126).std() * np.sqrt(252)

    # 칼럼 이름 변경
    df.columns = [ticker, ticker+'수정주가', '수익율', ticker+'변동성']
    return df
  

# UA_df와 KR code가 주어지면 기초자산의 변동성과 상관계수 추이를 보여주는 함수 작성
def vol_corr_from_krcode(file,UA_df,krcode,duration):
  # 20240727: 변환전 tickers로 수정
  # 앞에서 만들어진 UA_df를 입력값을 가져옴
  import pandas as pd
  import numpy as np

  uas=UA_df[krcode].iloc[1:4]
  tickers=[]
  for ua in uas:
    tickers.append(ua) # 변환되기 전 기초자산 이름을 tickers로
  issued_date=krcode2issuedate(file,kr_code)
  result_df = vol_corr(tickers, issued_date, duration)
  return result_df,tickers,issued_date


# 2,3종목에 대한 상관계수 분석까지
def vol_corr(tickers,date,duration):
    # 20240728:기초자산가격함수 수정
    # 결측값을 이전 값으로

    import pandas as pd
    import numpy as np

    result_df = pd.DataFrame()

    for ticker in tickers:
        df = get_close_price_btn(uname2yahoo(ticker), date, duration)
        if df is not np.nan:
            result_df = pd.concat([result_df, df], axis=1)

    if len(tickers) >= 2:
        # 결측값을 이전 값으로 채우기
        result_df.ffill(inplace=True)


        # 6개월 상관계수 계산
        returns_df = result_df[[uname2yahoo(ticker)[0] for ticker in tickers]].pct_change()
        rolling_corr = returns_df.rolling(window=126).corr()

        # 상관계수 추출 및 칼럼 추가
        for i in range(len(tickers)):
            for j in range(i + 1, len(tickers)):
                corr = rolling_corr.loc[(slice(None), uname2yahoo(tickers[i])[0]), uname2yahoo(tickers[j])[0]].reset_index(level=1, drop=True)
                result_df[f'{uname2yahoo(tickers[i])[0]}_{uname2yahoo(tickers[j])[0]}_corr'] = corr

    return result_df


# 새로운 데이터프레임으로 정리하기
def create_combined_df(result_df, tickers):
    import pandas as pd
    combined_df = pd.DataFrame(index=result_df.index)

    # 종가 데이터 추가
    for ticker in tickers:
        combined_df[ticker] = result_df[ticker]

    # 수정주가 데이터 추가
    for ticker in tickers:
        combined_df[f'{ticker}_수정주가'] = result_df[f'{ticker}수정주가']

    # 변동성 데이터 추가
    for ticker in tickers:
        combined_df[f'{ticker}_변동성'] = result_df[f'{ticker}변동성']

        # 상관계수 칼럼 추가
    if len(tickers) == 2:
        combined_df[result_df.columns[-1]] = result_df.iloc[:, -1]
    elif len(tickers) == 3:
        combined_df[result_df.columns[-3]] = result_df.iloc[:, -3]
        combined_df[result_df.columns[-2]] = result_df.iloc[:, -2]
        combined_df[result_df.columns[-1]] = result_df.iloc[:, -1]


    return combined_df

def remove_consecutive_zeros(df):
    import pandas as pd
    # 첫 번째 행이 모두 0인 경우 삭제
    if (df.iloc[0] == 0).all():
        df.drop(df.index[0], inplace=True)
        # 재귀적으로 함수 호출하여 다음 첫 번째 행이 모두 0인 경우도 확인하고 삭제
        remove_consecutive_zeros(df)

############################################################
# KR code에 따른 발행일 정보 가져오기
############################################################

def krcode2issuedate(file,kr_code):
  import pandas as pd
  df_issue_date=pd.DataFrame()
  df=pd.read_excel(file,skiprows=7,usecols=None)
  KR_Code=df.iloc[0,2:] # 발행코드 정보를 읽어옴
  Issue_Dates=df.iloc[6,2:] # 발행일자 정보를 읽어옴
  Issue_Dates=pd.to_datetime(Issue_Dates, format='%Y%m%d') # 날짜 형식 변경
  df_issue_date['KR_Code']=KR_Code
  df_issue_date['Issue_Dates']=Issue_Dates
  return df_issue_date[df_issue_date['KR_Code']==kr_code]['Issue_Dates'].dt.strftime('%Y-%m-%d').tolist()[0]


def uname2yahoo(name):
    # 2024년 7월28일 수정
    # KOSDAQ 150 INDEX 추가(20240726)
    ticker_diction={'I.GSPC':'^GSPC',
                   'SX5E INDEX':'^STOXX50E',
                   'I.N225':'^N225', #Nikkei 225 Index
                   'I.101':'^KS200', # KOSPI200 지수
                   'I.HSCE':'^HSCE', #Hang Seng China Enterprises Index
                    'TSLA US EQUITY':'TSLA',
                    'NVDA US EQUITY':'NVDA', # NVIDIA Corporation의 주식
                    'AMD US EQUITY':'AMD', # Advanced Micro Devices
                    'AMZN US EQUITY':'AMZN',
                    'INTC US EQUITY':'INTC', # intel corportion
                    'MU US EQUITY':'MU', #micron technology
                    'NDX INDEX':'^NDX',# NASDAQ-100 Index
                    'NFLX US EQUITY':'NFLX', #Netflix, Inc
                    'KR7051910008':'051910.KS', # LG화학, 확인
                    'KR7000660001':'000660.KS', # SK 하이닉스, 확인
                    'KR7005930003':'005930.KS', # 삼성전자, 확인
                    'KR7035420009':'035420.KS', # 네이버, 확인
                    'KR7066570003':'066570.KS', # LG전자, 확인
                    'KR7005490008':'005490.KS', # 포스코 홀딩스, 확인
                    'KR7034220004':'034220.KS', # LG디스플레이, 확인
                    'SPXESUP INDEX':'^SPXESUP', # S&P 500 Equal Weight Utilities Index
                    'KR7028260008':'028260.KS', # 삼성 C&T corporation, 확인
                    'AAPL US EQUITY':'AAPL', # Apple
                    'FB US EQUITY':'META', # Meta Platforms, Inc. (이전에는 Facebook, Inc.)의 주식
                    'QCOM UW EQUITY':'QCOM', # Qualcomm Incorporated의 주식
                    'I.GDAXI':'^GDAXI', # DAX (Deutscher Aktienindex)
                    'I.HSI':'^HSI', # Hang Seng Index
                    'KR7105560007':'105560.KS', # KB금융그룹, 확인
                    'SBUX UW EQUITY':'SBUX', # 스타벅스
                    'KR7251270005':'251270.KS', # 넷마블, 확인
                    'KR7207940008':'207940.KS', # 삼성바이오로직스, 확인
                    'KR7086790003':'086790.KS', # 하나금융지주, 확인
                    'KR7015760002':'015760.KS', # 한국전력공사, 확인
                    'KR7316140003':'316140.KS', # 우리금융지주, 확인
                    'KR7012330007':'012330.KS', # 현대모비스, 확인
                    'BA US EQUITY':'BA', # The Boeing Company
                    'XOM US EQUITY':'XOM', # Exxon Mobil Corporation
                    'KR7005380001':'005380.KS', # 현대자동차, 확인
                    'KR7090430000':'090430.KS', # 아모레퍼시픽, 확인
                    'KR7055550008':'055550.KS', # 신한파이낸셜 그룹, 확인
                    'KR7068270008':'068270.KS', # 셀트리온, 확인
                    'HSTECH INDEX':'HSTECH.HK', # Hang Seng TECH Index, 기간데이터 미제공인듯
                    'MSFT US EQUITY':'MSFT', # Microsoft Corporation
                    'KR7035720002':'035720.KS', # 카카오, 확인
                    'KR7000270009':'000270.KS', # 기아, 확인
                    'KR7009150004':'009150.KS', # 삼성전기, 확인
                    'KOSPI2LG INDEX':'KOSPI2LG.KS', # KOSPI 200 Large Cap Index, 어떤 기초자산인지 미확인
                    'BSK-SKTELECOM':'017670.KS', # SK텔레콤
                    'KR7139480008':'139480.KS', # E마트,확인
                    'C US EQUITY':'C', # Citigroup Inc.
                    'DIS US EQUITY':'DIS', # The Walt Disney Company
                    'KR7034730002':'034730.KS', # (주)SK,확인
                    'KR7006400006':'006400.KS', # 삼성SDI, 확인
                    'KR7017670001':'017670.KS', # SK텔레콤,확인
                    'NKE UN EQUITY':'NKE', # Nike, Inc.의 주식
                    'SPESG INDEX':'^SPESG', # S&P 500 ESG Index, 기간 데이터 미제공인 듯
                    'SX5EESG INDEX':'SX5EESG.SW', # EURO STOXX 50 ESG Index
                    'GM UN EQUITY':'GM', # General Motors Company
                    'KR7036570000':'036570.KS', # 엔씨소프트, 확인
                    'KR7096770003':'096770.KS', # SK이노베이션, 확인
                    'KR7018260000':'018260.KS', # 삼성SDS, 확인
                    'XIN0I INDEX':'XIN0.FGI', # FTSE China 50 Index
                    'KR7033780008':'033780.KS', # KT&G, 확인
                    'AMAT US EQUITY':'AMAT', # Applied Materials, Inc.의 주식
                    'META US EQUITY':'META', # Meta Platforms, Inc. (이전 Facebook, Inc.)의 주식
                    'KR7032830002':'032830.KS', # 삼성생명, 확인
                    'KR7024110009':'024110.KS', # 산업은행, 확인
                    'NVDA UW EQUITY':'NVDA', # NVIDIA Corporation의 주식
                    'KR7000810002':'000810.KS', # 삼성화재, 확인
                    'KR7003550001':'003550.KS', # (주) LG, 확인
                    'GS US EQUITY':'GS', # Goldman Sachs Group, Inc.
                    'TSM US EQUITY':'TSM', # Taiwan Semiconductor Manufacturing Company Limited (TSMC) 주식
                    'BAC US EQUITY':'BAC', # Bank of America Corporation
                    'GOOGL US EQUITY':'GOOGL', #  Alphabet Inc.의 주식
                    'KR7003670007':'003670.KS', # Posco Future M Co., Ltd.
                    'UBER US EQUITY':'UBER', # UBER 미국 주식
                    'PYPL US EQUITY':'PYPL', # Paypal Inc.
                    'KR7373220003':'373220.KS', # LG 에너지 솔루션
                   }
    ticker_diction_krx={'KOSDAQ 150 INDEX':'003280 ',# KOSDAQ 150 INDEX로서 pykrx의 티커로 대체
                        } 
    if name in ticker_diction.keys():
        return ticker_diction[name],'yahoo'
    elif name in ticker_diction_krx.keys():
        return ticker_diction_krx[name],'krx'
    else:
        print(f'{name}라는 티커가 없어 기타로 분류합니다')
        return '기타'

In [30]:
# 새로 만든 파일 경로 설정
import pandas as pd
import numpy as np

file=  r'C:\Temp\2022_KRW_하반기.xlsx'


# df_Daily_NAV 만들기


df=pd.read_excel(file,skiprows=32,usecols=None)

df_Daily_NAV=df.iloc[2:,2:] # Daily NAVs 만을 발췌
df_Daily_NAV=df_Daily_NAV.fillna(0) # NaN을 0으로 바꾸기
df_Daily_NAV = df_Daily_NAV.apply(pd.to_numeric, errors='coerce') # 모든 열을 숫자로 변환

# 연속된 0인 행 삭제
remove_consecutive_zeros(df_Daily_NAV)

# 일자를 인덱스 정보로 불러오고 포맷변경
Biz_Dates=df.iloc[-len(df_Daily_NAV):,1]
Biz_Dates=pd.to_datetime(Biz_Dates, format='%Y%m%d')

# ELS 발행정보 정보 읽어오기
df_Issuance_Info=pd.read_excel(file, nrows=35) # 처음 35행만 읽어오기
Notional_Amount = df_Issuance_Info.iloc[12,2:] # 발행총액 정보를 읽어옴
Notional_Amount = Notional_Amount.apply(int) # 숫자 데이터로 변환
# Series 이름제거
Notional_Amount = Notional_Amount.reset_index(drop=True)
KR_Code=df_Issuance_Info.iloc[7,2:] # 발행코드 정보를 읽어옴

# 날짜를 인덱스로 설정하기
df_Daily_NAV = df_Daily_NAV.set_index(Biz_Dates)
df_Daily_NAV.index.name = 'Biz_Dates'

df_Daily_NAV = df_Daily_NAV.rename(columns=KR_Code) #칼럼 이름 주기



In [31]:
###########################################################
# KR code에 존재하는 기준가, 기초자산명 확인
############################################################

kr_code='KR6KB0003V81'

df=pd.read_excel(file,skiprows=7,usecols=None)
KR_Code=df.iloc[0,2:] # 발행코드 정보를 읽어옴
kijun_date=Biz_Dates.iloc[-1].strftime('%Y-%m-%d') # 자료의 마지막 날짜로 설정

# 기준가 식별 데이터프레임
# 특정행만 추출
selected_rows = df.iloc[[5] + list(range(21, 25))]

# 새로운 데이터프레임으로 저장
kijun_df = selected_rows.iloc[:,2:].copy()
kijun_df.columns=KR_Code
kijun_df.columns.name='KR code'

# 기초자산 개수 구하기
nan_counts = kijun_df.iloc[1:5].notna().sum()
# 새로운 행에 NaN이 아닌 값의 개수를 추가
nan_counts_df = pd.DataFrame([nan_counts], index=['기초자산갯수'])
# 기존 데이터프레임에 새로운 행 추가
kijun_df = pd.concat([kijun_df.iloc[:5], nan_counts_df])
kijun_df.index=['발행금액','기초자산1','기초자산2','기초자산3','기초자산4','기초자산갯수']

# '발행금액' 행을 numeric으로 변환
kijun_df.loc['발행금액'] = pd.to_numeric(kijun_df.loc['발행금액'], errors='coerce')
kijun_df.loc['기초자산1'] = pd.to_numeric(kijun_df.loc['기초자산1'], errors='coerce')
kijun_df.loc['기초자산2'] = pd.to_numeric(kijun_df.loc['기초자산2'], errors='coerce')
kijun_df.loc['기초자산3'] = pd.to_numeric(kijun_df.loc['기초자산3'], errors='coerce')
kijun_df.loc['기초자산4'] = pd.to_numeric(kijun_df.loc['기초자산4'], errors='coerce')

# 특정행만 추출
selected_rows = df.iloc[[5] + list(range(17, 21))]

# 새로운 데이터프레임으로 저장
UA_df = selected_rows.iloc[:,2:].copy()
UA_df.columns=KR_Code
UA_df.columns.name='KR code'

# 기초자산 개수 구하기
nan_counts = UA_df.iloc[1:5].notna().sum()
# 새로운 행에 NaN이 아닌 값의 개수를 추가
nan_counts_df = pd.DataFrame([nan_counts], index=['기초자산갯수'])

# 기존 데이터프레임에 새로운 행 추가
UA_df = pd.concat([UA_df.iloc[:5], nan_counts_df])
UA_df.index=['발행금액','기초자산1','기초자산2','기초자산3','기초자산4','기초자산갯수']

# '발행금액' 행을 numeric으로 변환
UA_df.loc['발행금액'] = pd.to_numeric(UA_df.loc['발행금액'], errors='coerce')


In [32]:
kr_code='KR6KB0003V81'
krcode2issuedate(file,kr_code)

# 특정 코드의 처음 3개의 기초자산 정보만 불러오기
uas=UA_df[kr_code].iloc[1:4]
tickers=[]
for ua in uas:
  tickers.append(ua)

tickers

['I.GSPC', 'I.N225', 'SX5E INDEX']

In [38]:
import pandas as pd
import numpy as np

result_df = pd.DataFrame()

for ticker in tickers:
    df = get_close_price_btn(uname2yahoo(ticker), date, 1)
    if df is not np.nan:
        result_df = pd.concat([result_df, df], axis=1)

In [40]:
result_df

,^GSPC,^GSPC수정주가,수익율,^GSPC변동성,^N225,^N225수정주가,수익율,^N225변동성,^STOXX50E,^STOXX50E수정주가,수익율,^STOXX50E변동성
Date,,,,,,,,,,,,
2022-01-03,4796.560059,124.926685,NaN,NaN,NaN,NaN,NaN,NaN,4331.819824,114.186969,NaN,NaN
2022-01-04,4793.540039,124.848028,-0.000630,NaN,29301.789062,112.291054,NaN,NaN,4367.620117,115.130666,0.008264,NaN
2022-01-05,4700.580078,122.426881,-0.019393,NaN,29332.160156,112.407443,0.001036,NaN,4392.149902,115.777273,0.005616,NaN
2022-01-06,4696.049805,122.308889,-0.000964,NaN,28487.869141,109.171929,-0.028784,NaN,4324.810059,114.002191,-0.015332,NaN
2022-01-07,4677.029785,121.813512,-0.004050,NaN,28478.560547,109.136257,-0.000327,NaN,4305.830078,113.501878,-0.004389,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-19,NaN,NaN,NaN,NaN,33370.421875,127.882971,-0.009958,0.149748,4362.379883,114.992533,-0.007381,0.166035
2023-07-04,NaN,NaN,NaN,NaN,33422.519531,128.082621,-0.009801,0.146910,4390.990234,115.746704,-0.001628,0.153745
2023-09-04,NaN,NaN,NaN,NaN,32939.179688,126.230354,0.006987,0.155582,4279.870117,112.817572,-0.000647,0.158842


In [36]:
for ticker in tickers:
    df = get_close_price_btn(uname2yahoo(ticker), date, duration)
    if df is not np.nan:
        result_df = pd.concat([result_df, df], axis=1)

if len(tickers) >= 2:
    # 결측값을 이전 값으로 채우기
    result_df.ffill(inplace=True)


    # 6개월 상관계수 계산
    returns_df = result_df[[uname2yahoo(ticker)[0] for ticker in tickers]].pct_change()
    rolling_corr = returns_df.rolling(window=126).corr()

    # 상관계수 추출 및 칼럼 추가
    for i in range(len(tickers)):
        for j in range(i + 1, len(tickers)):
            corr = rolling_corr.loc[(slice(None), uname2yahoo(tickers[i])[0]), uname2yahoo(tickers[j])[0]].reset_index(level=1, drop=True)
            result_df[f'{uname2yahoo(tickers[i])[0]}_{uname2yahoo(tickers[j])[0]}_corr'] = corr

ValueError: Cannot set a DataFrame with multiple columns to the single column ^GSPC_^N225_corr

In [42]:
date = '2022-01-01'
duration = 1
vol_corr(tickers, date, duration)

,^GSPC,^GSPC수정주가,수익율,^GSPC변동성,^N225,^N225수정주가,수익율,^N225변동성,^STOXX50E,^STOXX50E수정주가,수익율,^STOXX50E변동성,^GSPC_^N225_corr,^GSPC_^STOXX50E_corr,^N225_^STOXX50E_corr
Date,,,,,,,,,,,,,,,
2021-01-04,3700.649902,77.643936,NaN,NaN,27258.380859,94.674404,NaN,NaN,3564.389893,82.775943,NaN,NaN,NaN,NaN,NaN
2021-01-05,3726.860107,78.193857,0.007083,NaN,27158.630859,94.327951,-0.003659,NaN,3547.850098,82.391839,-0.004640,NaN,NaN,NaN,NaN
2021-01-06,3748.139893,78.640332,0.005710,NaN,27055.939453,93.971281,-0.003781,NaN,3611.080078,83.860230,0.017822,NaN,NaN,NaN,NaN
2021-01-07,3803.790039,79.807936,0.014847,NaN,27490.130859,95.479324,0.016048,NaN,3622.419922,84.123576,0.003140,NaN,NaN,NaN,NaN
2021-01-08,3824.679932,80.246231,0.005492,NaN,28139.029297,97.733092,0.023605,NaN,3645.050049,84.649116,0.006247,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-20,3839.500000,80.557173,-0.002541,0.232895,25771.220703,89.509167,-0.007387,0.224037,3469.830078,80.579977,0.009123,0.273536,0.028547,0.388542,0.216012
2022-07-04,3839.500000,80.557173,-0.002541,0.232895,26153.810547,90.837987,0.008413,0.222278,3452.419922,80.175660,0.001192,0.274804,0.029978,0.389091,0.213359
2022-09-05,3839.500000,80.557173,-0.002541,0.232895,27619.609375,95.929031,-0.001129,0.206229,3490.010010,81.048616,-0.015340,0.248534,0.029376,0.388539,0.219836


In [43]:
file_out_ppt=r'C:\Temp\volcorr_from_krcode_0728.xlsx'

kr_code='KR6KB0003V81'

result_df,selected_tickers,issued_date=vol_corr_from_krcode(file,UA_df,kr_code,duration)

df_krcode=pd.DataFrame({'A':['KR code'],'B':[kr_code]})
df_issued_date = pd.DataFrame({'A':['발행일'],'B':[issued_date]})
df_tickers=pd.DataFrame({'A':'기초자산(Yahoo Tickers)','B':selected_tickers})


#############################################################
############## Vol and Corr from KR code ####################
#############################################################

with pd.ExcelWriter(file_out_ppt) as writer:
  df_volcorr=create_combined_df(result_df, selected_tickers)
  df_krcode.to_excel(writer, sheet_name='Historical Data from KRcode', startrow=0, startcol=0, index=False, header=None)
  df_issued_date.to_excel(writer, sheet_name='Historical Data from KRcode', startrow=2, startcol=0, index=False, header=None)
  df_tickers.to_excel(writer, sheet_name='Historical Data from KRcode', startrow=5, startcol=0, index=False, header=None)
  df_volcorr.to_excel(writer, sheet_name='Historical Data from KRcode',startrow=12,startcol=0)

IndexError: At least one sheet must be visible